In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
lda100 = pd.read_csv('../Data/Environmental Discourse/Full-TMs/lda100_loadings.csv', index_col=0)
for i in range(100):
    lda100['topic_{}'.format(i)] = lda100['topic_{}'.format(i)].apply(lambda x: x + 0.000001 if x == 0 else x)

In [4]:
lda100.shape

(46881, 106)

In [5]:
doc_topic = np.array(lda100[['topic_{}'.format(i) for i in range(100)]])

In [7]:
lda100['date'] = pd.to_datetime(lda100.date)
lda100['year'] = lda100.date.dt.year

In [19]:
lda100['year'] = lda100.date.dt.year

In [8]:
first_day = lda100.date.min()
lda100['date_index'] = lda100.date.apply(lambda x: (x - first_day) / np.timedelta64(1, 'D'))

In [48]:
future

array([], shape=(0, 100), dtype=float64)

In [ ]:
window = 365
novelties = []
transiences = []

def kld(a, b):
    return (a * np.log2(a/b)).sum()

for i in range(int(lda100.date_index.max())+1):
    current = doc_topic[lda100.date_index == i, :]
    past =    doc_topic[(lda100.date_index < i) & (i - lda100.date_index < window), :]
    future =  doc_topic[(lda100.date_index > i) & (lda100.date_index - i > window), :]
    
    for j in range(current.shape[0]):
        kld_before = []
        kld_after = []
        a = current[j, :]
        
        for k in range(past.shape[0]):
            b = past[k, :]
            kld_before.append(kld(a, b))
        
        for k in range(future.shape[0]):
            b = future[k, :]
            kld_after.append(kld(a, b))
            
        if kld_before == []:
            novelties.append(np.nan)
        else:
            novelties.append(np.mean(kld_before))
        if kld_after == []:
            transiences.append(np.nan)
        else:
            transiences.append(np.mean(kld_after))

In [ ]:
ntr = lda100.copy()[['url', 'title', 'date', 'year']]
ntr['novelty'] = novelties
ntr['transience'] = transiences
ntr['resonance'] = ntr.novelty - ntr.transience

In [40]:
ntr.transience.isna().sum()

46881

In [26]:
lda100.date.isna().sum()

0

In [21]:
novelties

[nan,
 nan,
 nan,
 -0.010886064692494984,
 0.02200635128108871,
 -0.531311103556467,
 -0.08528883346930957,
 0.1632371661691655,
 0.539150957444746,
 0.20559310445804502,
 0.3352180344576676,
 1.1623090312347932,
 0.39073226050301946,
 -0.4398207365870803,
 -0.2743274639461071,
 -0.5929494444727054,
 -0.690632526888265,
 -0.08518566099737417,
 -0.5202075612534975,
 -0.6906369949485398,
 -0.44990868052873106,
 -0.7009476449239777,
 -0.4187019638700578,
 -0.5894907678510881,
 1.4534910594059307,
 -0.7009927115698127,
 -0.07558133913974842,
 0.17083367592966517,
 -0.07730425095596064,
 -0.4425033853535513,
 -0.074840658251026,
 -0.3920527392256641,
 -0.2000678154971872,
 -0.48669870307559787,
 -0.2895991886132049,
 -0.32607921041454285,
 -0.9317376091227548,
 -0.4206008654797309,
 -0.32666081300591215,
 -0.6655785286331196,
 0.8567011983680684,
 0.0057385585938356225,
 0.04653365859385202,
 -0.30283045235413864,
 -0.3684753114962403,
 -0.653410267791693,
 -0.5329382656325787,
 0.102725745

In [11]:
np.mean([])

/home/mcpackard/anaconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mcpackard/anaconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [ ]:
lda100 = pd.read_pickle('../Data/Environmental Discourse/Full-TMs/lda100_loadings.pkl')

In [ ]:
lda100.head()

In [5]:
# Let's first try to set it up with a window size of 1 year, so dropping 2005 and 2021

In [9]:
first_day = lda100.date.min()
lda100['date_index'] = lda100.date.apply(lambda x: (x - first_day) / np.timedelta64(1, 'D'))

In [10]:
current = lda100[lda100.date_index == 400].sample(1)

In [11]:
before = lda100[(current.date_index.iloc[0] - lda100.date_index < 365) &
                (current.date_index.iloc[0] - lda100.date_index > 0)]

In [12]:
before_example = before.sample(1)

In [16]:
loadings = ['topic_{}'.format(i) for i in range(100)]
c = np.array(current[loadings])
b = np.array(before_example[loadings])

In [17]:
KLdivs = (c * np.log2(c/b)).sum(axis=1)

/tmp/ipykernel_6351/3722099466.py:1: RuntimeWarning: divide by zero encountered in true_divide
  KLdivs = (c * np.log2(c/b)).sum(axis=1)
/tmp/ipykernel_6351/3722099466.py:1: RuntimeWarning: invalid value encountered in true_divide
  KLdivs = (c * np.log2(c/b)).sum(axis=1)
/tmp/ipykernel_6351/3722099466.py:1: RuntimeWarning: divide by zero encountered in log2
  KLdivs = (c * np.log2(c/b)).sum(axis=1)
/tmp/ipykernel_6351/3722099466.py:1: RuntimeWarning: invalid value encountered in multiply
  KLdivs = (c * np.log2(c/b)).sum(axis=1)


In [20]:
c/b

/tmp/ipykernel_6351/3697610823.py:1: RuntimeWarning: divide by zero encountered in true_divide
  c/b
/tmp/ipykernel_6351/3697610823.py:1: RuntimeWarning: invalid value encountered in true_divide
  c/b


array([[       nan,        nan, 0.        ,        nan,        nan,
        0.94919446,        inf,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan, 0.        ,
               nan, 0.76598276,        nan,        nan,        nan,
               nan,        inf, 0.        ,        nan,        nan,
               inf,        inf,        nan, 3.01667699,        nan,
               inf,        nan,        nan,        nan,        inf,
               nan, 0.        ,        nan,        nan,        nan,
               nan,        inf,        inf,        nan, 0.        ,
               nan,        inf,        nan,        nan,        nan,
               nan,        nan,        inf,        nan,        nan,
               nan,        nan,        inf,        nan, 0.        ,
               nan,        nan, 0.        ,     

In [69]:
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))


kl_divergence(np.array(current[loadings]), np.array(before_example[loadings]))

/tmp/ipykernel_1854/3450576420.py:2: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(np.where(p != 0, p * np.log(p / q), 0))
/tmp/ipykernel_1854/3450576420.py:2: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(np.where(p != 0, p * np.log(p / q), 0))
/tmp/ipykernel_1854/3450576420.py:2: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.where(p != 0, p * np.log(p / q), 0))
/tmp/ipykernel_1854/3450576420.py:2: RuntimeWarning: invalid value encountered in multiply
  return np.sum(np.where(p != 0, p * np.log(p / q), 0))


inf

In [65]:
before.date.max()

Timestamp('2006-02-04 00:00:00')

In [39]:
example = lda100.sample(1, random_state=2)

In [45]:
lda100['in_window'] = lda100.date.apply(lambda x: x - example.date)

In [46]:
lda100

,source,url,title,date,text,year,topic_0,topic_1,topic_2,topic_3,...,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,example_date,date_diff,in_window
4,Resilience,https://www.resilience.org/stories/2021-07-12/...,The show is over,2021-07-12,"This piece was originally published on July 1,...",2021,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.028328,0.0,0.0,0.000000,0.0,2011-06-04,NaT,3691 days
5,Resilience,https://www.resilience.org/stories/2021-07-12/...,The Energy Bulletin Weekly 12 July 2021,2021-07-12,"Tom Whipple and Steve Andrews, Editors Quote o...",2021,0.000000,0.000000,0.012800,0.0,...,0.000000,0.000000,0.015830,0.0,0.0,0.000000,0.0,2011-06-04,NaT,3691 days
7,Resilience,https://www.resilience.org/stories/2021-07-12/...,On the Existence of A Man,2021-07-12,Philosophers and mystics throughout time have ...,2021,0.000000,0.000000,0.000000,0.0,...,0.000000,0.086977,0.000000,0.0,0.0,0.000000,0.0,2011-06-04,NaT,3691 days
9,Resilience,https://www.resilience.org/stories/2021-07-08/...,"Climate change, domination and the temporality...",2021-07-08,[Being masters or possesors of nature] has no ...,2021,0.000000,0.000000,0.000000,0.0,...,0.000000,0.122581,0.000000,0.0,0.0,0.000000,0.0,2011-06-04,NaT,3687 days
11,Resilience,https://www.resilience.org/stories/2021-07-07/...,Least worst politics,2021-07-07,In Parts I and II of A Small Farm Future I bui...,2021,0.000000,0.057836,0.000000,0.0,...,0.000000,0.269669,0.000000,0.0,0.0,0.000000,0.0,2011-06-04,NaT,3686 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95874,Grist,https://grist.org/article/the-digital-life/,The Digital Life,2013-11-03,Inventor and futurist Ray Kurzweil predicts th...,2013,0.000000,0.000000,0.010208,0.0,...,0.000000,0.000000,0.016374,0.0,0.0,0.000000,0.0,2011-06-04,NaT,883 days
95875,Grist,https://grist.org/article/2011-10-10-discarded...,Discarded glass bottles can be used to c...,2011-10-11,\tUnless you're stranded on a desert island...,2011,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,2011-06-04,NaT,129 days
95878,Grist,https://grist.org/article/at-least-the-citys-b...,At Least the City’s Back Up and … Oh,2006-01-31,Gulf Coast ecosystems slow to bounce back afte...,2006,0.000000,0.000000,0.000000,0.0,...,0.375563,0.000000,0.000000,0.0,0.0,0.000000,0.0,2011-06-04,NaT,-1950 days
95879,Grist,https://grist.org/climate-policy/2011-08-02-st...,Storm chaser: Sen. Durbin carries on lon...,2011-08-03,Sen. Dick Durbin. Cross-posted from ThinkProgr...,2011,0.047527,0.100274,0.041664,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.100116,0.0,2011-06-04,NaT,60 days


In [20]:
lda100['example_date'] = example1.date
lda100['date_diff'] = (lda100.date - lda100.example_date)

In [37]:
dates_df = pd.DataFrame({'date1':list(lda100.date.sample(10)), 
                         'date2':list(lda100.date.sample(1)) * 10})

In [38]:
dates_df['diff'] = dates_df.date1 - dates_df.date2
dates_df

,date1,date2,diff
0,2012-07-19,2019-02-07,-2394 days
1,2009-09-22,2019-02-07,-3425 days
2,2009-08-01,2019-02-07,-3477 days
3,2013-03-05,2019-02-07,-2165 days
4,2008-08-08,2019-02-07,-3835 days
5,2011-05-31,2019-02-07,-2809 days
6,2006-06-01,2019-02-07,-4634 days
7,2014-02-06,2019-02-07,-1827 days
8,2020-06-11,2019-02-07,490 days
9,2011-10-19,2019-02-07,-2668 days


In [27]:
example1.date

78482   2011-06-04
Name: date, dtype: datetime64[ns]

In [21]:
lda100.date_diff

4       NaT
5       NaT
7       NaT
9       NaT
11      NaT
         ..
95874   NaT
95875   NaT
95878   NaT
95879   NaT
95881   NaT
Name: date_diff, Length: 46881, dtype: timedelta64[ns]

78482   2011-06-04
Name: date, dtype: datetime64[ns]

In [5]:
import argparse
import os

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from lda import LDA

def learn_topics(textpath, topicnum):

    with open(textpath) as f:
        texts = f.readlines()

    # Get vocabulary and word counts.  Use the top 10,000 most frequent
    # lowercase unigrams with at least 3 alphabetical, non-numeric characters,
    # punctuation treated as separators.
    CVzer = CountVectorizer(token_pattern=r"(?u)\b[^\W\d]{3,}\b",
                            max_features=None,
                            lowercase=True)
    doc_vcnts = CVzer.fit_transform(texts)
    vocabulary = CVzer.get_feature_names()

    # Learn topics.  Refresh conrols print frequency.
    lda_model = LDA(topicnum, n_iter=8000, refresh=2000) 
    doc_topic = lda_model.fit_transform(doc_vcnts)
    topic_word = lda_model.topic_word_

    return doc_topic, topic_word, vocabulary

ModuleNotFoundError: No module named 'lda'